In [1]:
import os
import warnings
import shutil
import logging
import io
import zipfile
import neurom
import matplotlib
import json
import requests
import tarfile
from neurom import viewer
from ipywidgets.widgets import Select, Layout, Button, Image, Output, Label
from IPython.display import HTML, Javascript, IFrame, display, clear_output

warnings.filterwarnings("ignore")

In [2]:
%%capture
!pip install NEURON-nightly --upgrade
!rm -rf /opt/app-root/lib/nrn

In [3]:
# move to BPOPTANALYSIS folder
if os.path.isdir('BPOPTANALYSIS') == True:
    shutil.rmtree('BPOPTANALYSIS')
    
!mkdir BPOPTANALYSIS
os.chdir("BPOPTANALYSIS")

h0 = HTML("""
<style>
    .widget-label { min-width: 17ex !important;}
</style>
""")

js_code = """
<script type="text/Javascript">
    document.styleSheets[0].insertRule(".widget-listbox.form-control { width: auto }", 1);
    document.styleSheets[0].insertRule(".form_login > label { width: 30% }", 1);
    document.styleSheets[0].insertRule(".form_login > label + input { width: 70% }", 1);
</script>
"""

h1 = HTML(js_code)
h1.display = 'none'
display(h1)

In [4]:
%%capture
URL=('https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/hippocampus_optimization/bsp_data_repository-4.0.5_Python3.tar.gz')
!wget "{URL}"
!tar -zxvf bsp_data_repository-4.0.5_Python3.tar.gz

In [5]:
os.chdir('bsp_data_repository-4.0.5_Python3/optimizations')

FD = [x for x in os.listdir('.') if x not in ['README']]

for i in range(len(FD)):
    shutil.copytree(FD[i], os.path.join('..', '..', FD[i]))
    
os.chdir('../..')

In [6]:
logger = logging.getLogger()
logging.disable(logging.WARNING)

In [7]:
def plot_to_html(plt):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return "<img src='data:image/png;base64,{}'/>".format(base64.b64encode(buf.getvalue()).decode('ascii'))

In [8]:
FS = ['GitHub', 'Storage']
c1 = Select(description='Files are located on', options=FS, layout=Layout(height='100px', width='50%'), style={'description_width':'initial'}, value=FS[0])
display(c1)
list_fd = sorted(FD)
w3 = Select(description='optimizations', options=list_fd, layout=Layout(height='100px', width='50%'), value=list_fd[0])
display(w3)
message = Label("No zip or tar archive inside the chosen directory!")
display(message)
message.layout.display='none'

Select(description='Files are located on', layout=Layout(height='100px', width='50%'), options=('GitHub', 'Sto…

Select(description='optimizations', layout=Layout(height='100px', width='50%'), options=('CA1_int_bAC_011017HP…

Label(value='No zip or tar archive inside the chosen directory!')

In [9]:
os.chdir(w3.value)
for files in os.listdir('.'):
    if files.endswith('.zip'):
        zip_ref = zipfile.ZipFile(files, 'r')
        zip_ref.extractall()
        zip_ref.close() 

In [10]:
os.chdir(w3.value)
os.chdir('morphology')
for files in os.listdir('.'):
    if files.endswith(".asc"):
        nrn = neurom.load_neuron(files);

In [11]:
%matplotlib agg
plt, ax = viewer.draw(nrn, mode='2d')
plt.savefig("{}.png".format(w3.value))
plt.clear()

%matplotlib inline
file = open("{}.png".format(w3.value), "rb")
image = file.read()
out1 = Image(value=image, format='png', width=400, height=500)
display(out1)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x06\x00\x00\x005\xd1\xdc…

In [12]:
os.chdir('../../..')
out = Output()
display(out)

Output()

In [13]:
def on_w3_clicked(change):
    out1.layout.display = 'none'
    message.layout.display = 'none'
    with out:
        clear_output()
    %matplotlib agg
    BTV.disabled=True
    BTS.disabled=True
    if c1.value == 'GitHub':
        os.chdir(w3.value)
        for files in os.listdir('.'):
            if files.endswith('.zip'):
                zip_ref = zipfile.ZipFile(files, 'r')
                zip_ref.extractall()
                zip_ref.close() 
        os.chdir(w3.value)
        os.chdir('morphology')
        for files in os.listdir('.'):
            if files.endswith(".asc"):
                nrn = neurom.load_neuron(files);
        plt,ax=viewer.draw(nrn, mode='2d');
        for files in os.listdir('.'):
            if files.endswith('.png'):
                os.remove(files)
        plt.savefig("{}.png".format(w3.value))
        plt.clear()
        %matplotlib inline
        file = open("{}.png".format(w3.value), "rb")
        image = file.read()
        out1.value=image
        out1.layout.display=''
        os.chdir('../../..')    
    else:
        if len(w3.options) != 0:
            hdr = clb_oauth.get_token()
            my_url = "https://iam.ebrains.eu/auth/realms/hbp/protocol/openid-connect/userinfo"
            headers = {'Authorization': 'Bearer ' + hdr}
            res = requests.get(my_url, headers = headers)
            if os.path.isdir(w3.value) == False:
                os.mkdir(w3.value)
            os.chdir("..")
            for name in ['BluePyOptAll', 'BluePyOptOne']:
                path_to_dir = os.path.join(name, 'resultsNSG', str(res.json()['preferred_username']), w3.value)
                if os.path.exists(path_to_dir):
                    for f in [folder for folder in os.listdir(path_to_dir) if folder[0] != "."]:
                        shutil.copy2(os.path.join(path_to_dir, f), os.path.join("BPOPTANALYSIS", w3.value, f))
             
            for name in ['results', 'resultsPizdaint-SA']:
                path_to_dir = os.path.join(name, w3.value)
                if os.path.exists(path_to_dir):
                    for f in [folder for folder in os.listdir(path_to_dir) if folder[0] != "."]:
                        shutil.copy2(os.path.join(path_to_dir, f), os.path.join("BPOPTANALYSIS", w3.value, f))
                        
            os.chdir(os.path.join("BPOPTANALYSIS", w3.value))
            is_valid = True
            if any(f.endswith(".tar.gz") for f in os.listdir(".")):
                ff = tarfile.open("output.tar.gz")
                ff.extractall(os.path.join('..', 'OUTPUTBPOPT', w3.value))
                ff.close()
                os.chdir(os.path.join('..', 'OUTPUTBPOPT', w3.value))
            elif any(f.endswith(".zip") for f in os.listdir(".")):
                zip_ref = zipfile.ZipFile('output.zip', 'r')
                zip_ref.extractall(os.path.join('..', 'OUTPUTBPOPT', w3.value))
                zip_ref.close() 
                os.chdir(os.path.join('..', 'OUTPUTBPOPT', w3.value))
            else:
                is_valid = False
                message.layout.display=''
            if is_valid:
                for files in os.listdir('.'):
                    if os.path.isdir(files):
                        foldername1=files

                os.chdir(foldername1 + "/morphology")  
                for files in os.listdir('.'):
                    if files.endswith(".asc"):
                        nrn = neurom.load_neuron(files);
                plt,ax=viewer.draw(nrn, mode='2d');
                for files in os.listdir('.'):
                    if files.endswith('.png'):
                        os.remove(files)
                plt.savefig("{}.png".format(w3.value))
                plt.clear()
                %matplotlib inline
                file = open("{}.png".format(w3.value), "rb")
                image = file.read()
                out1.value=image
                out1.layout.display=''
                os.chdir('../../..')
            os.chdir("..")
    BTV.disabled=False
    
w3.observe(on_w3_clicked, names='value')

In [14]:
def on_c1_clicked(change):
    out1.layout.display='none'
    with out:
        clear_output()
    BTV.disabled=True
    BTS.disabled=True
    w3.layout.display='none'
    if c1.value == 'GitHub':
        w3.options = sorted(FD)
        w3.layout.display=''
        w3.value = w3.options[1]
        message.layout.display='none'
        BTV.disabled=False
    else:
        hdr = clb_oauth.get_token()
        headers = {'Authorization': 'Bearer ' + hdr}
        my_url = "https://iam.ebrains.eu/auth/realms/hbp/protocol/openid-connect/userinfo"
        res = requests.get(my_url, headers = headers)
        w3list = []
        
        for name in ['BluePyOptAll', 'BluePyOptOne']:
            path_to_dir = os.path.join("..", name, 'resultsNSG', str(res.json()['preferred_username']))
            if os.path.exists(path_to_dir):
                w3list.extend([folder for folder in os.listdir(path_to_dir) if folder[0] != "."])
        
        for name in ['results', 'resultsPizdaint-SA']:
            path_to_dir = os.path.join("..", name)
            if os.path.exists(path_to_dir):
                w3list.extend([folder for folder in os.listdir(path_to_dir) if folder[0] != "."])

        w3list2=[]
        for file in w3list:
            w3list2.append(file[file.find('/')+1:len(file)])

        w3list=w3list2
        if len(w3list) != 0:
            w3.options = sorted(w3list)
            w3.layout.display = ''
            w3.value = w3.options[1]
            BTV.disabled = False
        else:
            w3.options = []
            w3.layout.display = 'none'
            with out:
                clear_output()
            out1.layout.display = 'none'
                
c1.observe(on_c1_clicked, names='value')

In [15]:
def runanalysis2(_):
    BTV.disabled = True
    BTS.disabled = True
    if c1.value == 'GitHub':
        os.chdir(w3.value)
        data = json.load(open(w3.value + '_meta.json'))
        os.chdir(w3.value)
        os.chdir('figures')
        with out:
            display(
                IFrame(
                    os.path.join(
                        '.',
                        'BPOPTANALYSIS',
                        w3.value,
                        w3.value,
                        'figures',
                        'neuron_evolution' + '_' + data['best_run'][0:data['best_run'].find('.pkl')] + '.pdf'
                    ),
                    width=600,
                    height=300
                )
            )
            display(
                IFrame(
                    os.path.join(
                        '.',
                        'BPOPTANALYSIS',
                        w3.value,
                        w3.value,
                        'figures',
                        'neuron_objectives' + '_' + data['best_run'][0:data['best_run'].find('.pkl')] + '.pdf'
                    ),
                    width=600,
                    height=300
                )
            )
            display(
                IFrame(
                    os.path.join(
                        '.',
                        'BPOPTANALYSIS',
                        w3.value,
                        w3.value,
                        'figures',
                        'neuron_responses' + '_' + data['best_run'][0:data['best_run'].find('.pkl')]+'.pdf'
                    ),
                    width=600, 
                    height=300
                )
            )
        os.chdir('../../..')
        BTS.disabled = False
    else:
        os.chdir("OUTPUTBPOPT")
        if len(w3.options)!=0:
            os.chdir(w3.value)
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername1=files
            os.chdir(foldername1)
            if not(any(os.path.isdir(File) and File.endswith('figures') for File in os.listdir("."))):
                os.mkdir('figures')                
            os.chdir('figures')
            for files in os.listdir('.'):
                os.remove(files)
            os.chdir('../checkpoints')
            if not os.listdir('.'):
                BTS.disabled=True
                os.chdir('../../../..')
            else:
                if 'checkpoint.pkl' not in os.listdir('.'):
                    for files in os.listdir('.'):
                        if files.endswith('pkl'):
                            shutil.copy(files, 'checkpoint.pkl')
                os.chdir('..')
                if os.path.isdir('x86_64')==True:
                    shutil.rmtree('x86_64')
                if os.path.isdir('r_0')==True:
                    shutil.rmtree('r_0')
                os.mkdir('r_0')
                myvar = !nrnivmodl mechanisms
                myvar = !python opt_neuron.py --analyse --checkpoint ./checkpoints/
                os.chdir('figures')
                with out:
                    display(
                        IFrame(
                            os.path.join(
                                '.',
                                'BPOPTANALYSIS',
                                'OUTPUTBPOPT',
                                w3.value,
                                foldername1,
                                'figures',
                                'neuron_evolution_checkpoint_0-.pdf'
                            ),
                            width=600,
                            height=300
                        )
                    )
                    display(
                        IFrame(
                            os.path.join(
                                '.',
                                'BPOPTANALYSIS',
                                'OUTPUTBPOPT',
                                w3.value,
                                foldername1,
                                'figures',
                                'neuron_objectives_checkpoint_0-.pdf'
                            ),
                            width=600,
                            height=300
                        )
                    )
                    display(
                        IFrame(
                            os.path.join(
                                '.',
                                'BPOPTANALYSIS',
                                'OUTPUTBPOPT',
                                w3.value,
                                foldername1,
                                'figures',
                                'neuron_responses_checkpoint_0-.pdf'
                            ),
                            width=600,
                            height=300
                        )
                    )
                os.chdir('../../../..')
                BTS.disabled = False

In [16]:
def saveanalysis(_):
    BTV.disabled=True
    BTS.disabled=True
    storeto_path1 = os.path.join("BluePyOptAnalysis")
    if not os.path.exists(storeto_path1):
        os.mkdir(storeto_path1)
    hdr = clb_oauth.get_token()
    my_url = "https://iam.ebrains.eu/auth/realms/hbp/protocol/openid-connect/userinfo"
    headers = {'Authorization':'Bearer ' + hdr}
    res = requests.get(my_url, headers = headers)
    storeto_path2 = os.path.join(storeto_path1, str(res.json()['preferred_username']))
    if not os.path.exists(storeto_path2):
        os.mkdir(storeto_path2)        
    storeto = os.path.join(storeto_path2,w3.value)
    if not os.path.exists(storeto):
        os.mkdir(storeto)
        if c1.value=='GitHub':
            os.chdir(w3.value)
            data = json.load(open(w3.value + '_meta.json'))
            os.chdir(w3.value + "/figures")
            f = str('neuron_evolution' + '_' + data['best_run'][0:data['best_run'].find('.pkl')] + '.pdf')
            shutil.copy2(f, os.path.join("../../..", storeto, f))
            f = str('neuron_objectives' + '_' + data['best_run'][0:data['best_run'].find('.pkl')] + '.pdf')
            shutil.copy2(f, os.path.join("../../..", storeto, f))
            f = str('neuron_responses' + '_' + data['best_run'][0:data['best_run'].find('.pkl')] + '.pdf')
            shutil.copy2(f, os.path.join("../../..", storeto, f))
            os.chdir('../checkpoints')
            f = str('cell' + '_' + data['best_run'][0:data['best_run'].find('.pkl')] + '_0' + '.hoc')
            shutil.copy2(f, os.path.join("../../..", storeto, f))
            os.chdir('../../..')
        else:
            os.chdir(os.path.join("OUTPUTBPOPT", w3.value))
            for files in os.listdir('.'):
                if os.path.isdir(files):
                    foldername1=files
            os.chdir(os.path.join(foldername1, 'figures'))
            f = 'neuron_evolution_checkpoint_0-.pdf'
            shutil.copy2(f, os.path.join("../../../..", storeto, f))
            f = 'neuron_objectives_checkpoint_0-.pdf'
            shutil.copy2(f, os.path.join("../../../..", storeto, f))
            f = 'neuron_responses_checkpoint_0-.pdf'
            shutil.copy2(f, os.path.join("../../../..", storeto, f))
            os.chdir('../checkpoints')
            f = 'cell_checkpoint_0.hoc'
            shutil.copy2(f, os.path.join("../../../..", storeto, f))
            os.chdir('../../../..')

In [17]:
BTV = Button()
BTV.description = 'View analysis'
BTV.background_color = 'gainsboro'
BTV.border_color = 'black'
BTV.on_click(runanalysis2)
display(BTV)

BTS = Button()
BTS.description = 'Save analysis'
BTS.background_color = 'gainsboro'
BTS.border_color = 'black'
BTS.on_click(saveanalysis)
BTS.disabled=True
display(BTS)

Button(description='View analysis', style=ButtonStyle())

Button(description='Save analysis', disabled=True, style=ButtonStyle())

In [18]:
from IPython.display import HTML

HTML('''
<button class="toggle_button" onclick="toggleCode(this)">Show Code</button>
<script>
    function toggleCode(button) {
        boxes = button.closest(".jp-Notebook").querySelectorAll(".jp-Cell-inputWrapper");
        if (boxes[0].style.display == "none") {
            boxes.forEach(box => box.style.display = '');
            button.innerHTML = "Hide Code";
        } else {
            boxes.forEach(box => box.style.display = 'none');
            button.innerHTML = "Show Code";
        }
    }
    document.querySelectorAll(".jp-Cell-inputWrapper").forEach(box => box.style.display = 'none');
</script>
''')